In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics  import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV


dataset_path = 'BMWclass_data.csv'
df = pd.read_csv('BMWclass_data.csv')

# Extract dataset name from the file path
dataset_name = dataset_path.split('/')[-1].split('.')[0] 


In [7]:
df.head

<bound method NDFrame.head of             Date       Open       High        Low      Close  Adj Close  \
0     2009-12-29  15.290000  15.380000  15.280000  15.370000   8.740845   
1     2009-12-30  15.200000  15.300000  15.150000  15.200000   8.644167   
2     2010-01-04  15.510000  15.510000  15.350000  15.350000   8.729471   
3     2010-01-05  15.340000  15.600000  15.340000  15.600000   8.871645   
4     2010-01-06  15.600000  15.860000  15.600000  15.850000   9.013818   
...          ...        ...        ...        ...        ...        ...   
2086  2022-12-16  29.299999  29.370001  28.980000  29.120001  26.865105   
2087  2022-12-22  29.510000  29.510000  29.180000  29.420000  27.141874   
2088  2022-12-28  29.559999  29.559999  29.270000  29.290001  27.021942   
2089  2022-12-29  29.620001  29.840000  29.620001  29.799999  27.492449   
2090  2022-12-30  29.770000  29.770000  29.450001  29.660000  27.363289   

      Volume      MACD        RSI      MA        EMA     SMA       ST

In [8]:


# Load your OHLC dataset with indicators
# Replace 'your_dataset.csv' with the actual file path or DataFrame variable


# Assuming 'PatternClass' column is already defined in your dataset

# Remove rows where PatternClass is 0 (No Pattern)
df = df[df['PatternClass'] != 0]

# Convert 'Date' to datetime and extract relevant features
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# Select features (X) and target variable (y)
X = df.drop(['PatternClass', 'Date', 'Open', 'High', 'Low', 'Close','Volume','Adj Close'], axis=1)
selected_features = ['PatternClass', 'Open', 'High', 'Low', 'Close','Volume','Adj Close']  # Replace with your actual feature names
X = df[selected_features]

y = df['PatternClass']
print("Feature Columns:", X.columns)

# Handle missing values by filling with mean
X = X.fillna(X.mean())

# Check for and handle infinite values
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X = X.fillna(0)  # You can customize the filling strategy based on your data

# Encode any categorical variables if necessary
label_encoder = LabelEncoder()
X_encoded = X.apply(lambda col: label_encoder.fit_transform(col.astype(str)) if col.dtype == 'O' else col)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=2)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the classifiers
rf_classifier = RandomForestClassifier(random_state=100, n_estimators=100)
svm_classifier = SVC(random_state=100)
logreg_classifier = LogisticRegression(random_state=100)
dt_classifier = DecisionTreeClassifier(random_state=100)
knn_classifier = KNeighborsClassifier()
nb_classifier = GaussianNB()
extra_trees_classifier = ExtraTreesClassifier(random_state=100, n_estimators=100)  # Added Extra Trees Classifier

# List of classifiers
classifiers = [rf_classifier, svm_classifier, logreg_classifier, dt_classifier, knn_classifier, nb_classifier, extra_trees_classifier]
classifier_names = ['Random Forest', 'SVM', 'Logistic Regression', 'Decision Tree', 'KNN', 'Naive Bayes', 'Extra Trees']

# Initialize DataFrame to store results
results_df = pd.DataFrame(columns=['Classifier', 'Accuracy', 'F1 Score'])

# Loop through each classifier
for classifier, classifier_name in zip(classifiers, classifier_names):
    # Train the classifier
    classifier.fit(X_train_scaled, y_train)

    # Make predictions on the test set
    y_pred = classifier.predict(X_test_scaled)

   # Evaluate the classifier
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')

    # Append results to the DataFrame
    results_df = results_df.append({'Classifier': classifier_name, 'Accuracy': accuracy, 'F1 Score': f1,'Precision': precision, 'Recall': recall}, ignore_index=True)

# Display the results and name of dataset
print(f"Dataset Name: {dataset_name}")
print()
print(results_df)



Feature Columns: Index(['PatternClass', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close'], dtype='object')


C:\Users\ASUS\AppData\Local\Temp\ipykernel_9484\377009696.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Classifier': classifier_name, 'Accuracy': accuracy, 'F1 Score': f1,'Precision': precision, 'Recall': recall}, ignore_index=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_9484\377009696.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Classifier': classifier_name, 'Accuracy': accuracy, 'F1 Score': f1,'Precision': precision, 'Recall': recall}, ignore_index=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

Dataset Name: BMWclass_data

            Classifier  Accuracy  F1 Score  Precision    Recall
0        Random Forest  0.993631  0.993282   0.994290  0.993631
1                  SVM  0.990446  0.990275   0.991769  0.990446
2  Logistic Regression  0.971338  0.957454   0.944412  0.971338
3        Decision Tree  1.000000  1.000000   1.000000  1.000000
4                  KNN  0.966561  0.958855   0.966762  0.966561
5          Naive Bayes  1.000000  1.000000   1.000000  1.000000
6          Extra Trees  0.995223  0.995054   0.995423  0.995223


C:\Users\ASUS\AppData\Local\Temp\ipykernel_9484\377009696.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Classifier': classifier_name, 'Accuracy': accuracy, 'F1 Score': f1,'Precision': precision, 'Recall': recall}, ignore_index=True)
